<a href="https://colab.research.google.com/github/dataxalco/Whatsapp-data-extraction/blob/main/CaregiverRotaToClientSchedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instructions**

This notebook reformats a caregiver rota to per-client schedules displayed in
a calendar view.

## **1. Import Dependencies**

In [7]:
import pandas as pd
import calendar
import gspread
from google.colab import auth, drive
from google.auth import default
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Font, Border, Side

# Authenticate and create the PyDrive client
auth.authenticate_user()

# Connect to Google Sheets
gc = gspread.authorize(default()[0])

## **2. User Input and Customization**

You need to open the code below to input your desired values. For each input, either enter a value when prompted or hit "Enter" to accept the default.

To find your Google Sheet Id, go to your Google Sheet, and in the URL, find the sheet_id:

  *   Example URL: https://docs.google.com/spreadsheets/d/your_sheet_id/edit#gid=0

  *   The sheet_id is the part that comes after /d/ and before /edit.

In [8]:
# Configure Google sheet
sheet_id = input("Enter the sheet id of the caregiver Rota: ").strip() or "1E_2kTuIfz6QVFR1b5GhTv9hB5hzYI0vM2pqly1vLNPo"
sheet_name = input("Enter the sheet name ('Rota' by default): ").strip() or "Rota"
output_excel_path = input(
    "Choose the name of the Excel file that will be created (Combined_CaregiverRota.xlsx by default): ").strip() or "Combined_CaregiverRota.xlsx"

# Configure date and time
now = datetime.now()
year = int(input("Enter the year you're planning for (current year by default): ").strip() or now.year)
month = int(input("Enter the month you're planning for (e.g. 8, current month by default): ").strip() or now.month)
day = int(input("Enter the day of the month you want to start with (e.g. 19, 1 by default): ").strip() or 1)
num_days = calendar.monthrange(year, month)[1]
first_day = datetime(year, month, day).weekday()

Enter the sheet id of the caregiver Rota: 1E_2kTuIfz6QVFR1b5GhTv9hB5hzYI0vM2pqly1vLNPo
Enter the sheet name ('Rota' by default): Rota
Choose the name of the Excel file that will be created (Combined_CaregiverRota.xlsx by default): test.xlsx
Enter the year you're planning for (current year by default): 2024
Enter the month you're planning for (e.g. 8, current month by default): 8
Enter the day of the month you want to start with (e.g. 19, 1 by default): 1


## **3. Load the Rota**

In [9]:
# Open Google Sheet using the Google Sheet ID
sheet = gc.open_by_key(sheet_id)

# Select the first sheet (worksheet) in the Google Sheet
worksheet = sheet.worksheet(sheet_name)

# Convert the worksheet to a pandas DataFrame
df = pd.DataFrame(worksheet.get('A5:ZZ'))

## **4. Define a helper function to build the new spreadsheet**

In [10]:
def build_sheet(row, day):
    """
    Save a DataFrame to a CSV file.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data to save.
    filename (str): The name of the CSV file to save the data into.

    Returns:
    None
    """
    data = []
    columns = ['', '', '', row.iloc[2], '', '', '', '']
    days_of_the_week = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    # The start of the shift schedule is in column 10.
    col = 10
    for i in range(6):
        line = [""]
        dayshift = ["Dayshift"]
        nightshift = ["Nightshift"]
        data.append(days_of_the_week)
        for j in range(7):
            if i == 0 and j < first_day:
                line.append("")
                dayshift.append("")
                nightshift.append("")
            elif day > num_days:
                line.append("")
                dayshift.append("")
                nightshift.append("")
            else:
                line.append(calendar.month_name[month] + "-" + str(day))
                dayshift.append(str(row.iloc[col]).split(' ')[0])
                nightshift.append(str(row.iloc[col+1]).split(' ')[0])
                day += 1
                col += 2
        data.append(line)
        data.append(dayshift)
        data.append(nightshift)
        if day > 31:
            break

    return pd.DataFrame(data, columns=columns)

## **5. Create the output excel file**

In [11]:
# Create an Excel writer to save all sheets into one Excel file
# First, write the data to the Excel file using Pandas
with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Extract the patient/client name (assuming it's in the first column)
        patient_name = row.iloc[2]
        if pd.isna(patient_name):
            continue
        if patient_name.strip() == "":
            break

        # Clean the patient name to use it as a sheet name
        sheet_name = f"{patient_name.replace('/', '_').replace(' ', '_')}"

        # Convert the single row to a DataFrame for writing it to a sheet
        patient_df = build_sheet(row, day)

        # Write the DataFrame to a new sheet in the Excel file
        patient_df.to_excel(writer, sheet_name=sheet_name, index=False)

## **6. Adjust the formatting on the new excel file**

In [12]:
# Now, open the Excel file and apply formatting
wb = load_workbook(output_excel_path)

for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]

    # Apply bold formatting to the second row (index 2) of each sheet
    for cell in ws[2]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[3]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[6]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[7]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[10]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[11]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[14]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[15]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[18]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws[19]:
        cell.font = Font(bold=True, italic=True)
    for cell in ws['A']:
        cell.font = Font(bold=True)
    # Resize the columns
    for col in ws.columns:
        column = col[0].column_letter  # Get the column name
        ws.column_dimensions[column].width = 12

    # Define a border with no lines (effectively removing the border)
    no_border = Border(left=Side(border_style=None),
                   right=Side(border_style=None),
                   top=Side(border_style=None),
                   bottom=Side(border_style=None))

    # Remove borders from specific columns, for example, columns A and B
    for cell in ws[1]:
        cell.border = no_border

# Save the workbook with the formatting applied
wb.save(output_excel_path)

print(f"Created Excel file with all patient sheets: {output_excel_path}")

Created Excel file with all patient sheets: test.xlsx


## **7. Copy the new file to My Drive**

In [13]:
# Save to Google Drive
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Move the file to your Google Drive
!cp {output_excel_path} /content/drive/My\ Drive/

Mounted at /content/drive
